# Bayes Course Pre-Work

As a first step, import some of the libraries we will be using in the course. This will test that everything is installed correctly on your system. Running the cell below should not return errors.

In [1]:
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
import pandas as pd
import pymc3 as pm
assert pm.__version__ == '3.9.2'
import sklearn as sk
import seaborn as sns
import theano.tensor as tt
import theano

print('All packages imported successfully!')

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\Zhekun Xiong\AppData\Local\Continuum\Anaconda3\envs\bayes_course\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


"dot" with args ['-Tps', 'C:\\Users\\ZHEKUN~1\\AppData\\Local\\Temp\\tmprn4yy9cs'] returned code: 1

stdout, stderr:
 b''
b"'C:\\Users\\Zhekun' is not recognized as an internal or external command,\r\noperable program or batch file.\r\n"



WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


All packages imported successfully!


If everything was imported, you are ready to work through the exercises below.

## Data Import and Manipulation

Loading and processing data has become easier since the development of the `pandas` library, which provides data structures and functions for automating key data operations. 

For importing data from most common storage formats, several `read_*` functions are available.

In [2]:
[f for f in dir(pd) if f.startswith('read_')]

['read_clipboard',
 'read_csv',
 'read_excel',
 'read_feather',
 'read_fwf',
 'read_gbq',
 'read_hdf',
 'read_html',
 'read_json',
 'read_orc',
 'read_parquet',
 'read_pickle',
 'read_sas',
 'read_spss',
 'read_sql',
 'read_sql_query',
 'read_sql_table',
 'read_stata',
 'read_table']

The `../data/` subdirectory includes some clinical trial data in `cdystonia.csv`. Choose the appropriate function and use it to import this data to a variable called `cdystonia`. Print the first 15 lines of the resulting `DataFrame`.

In [28]:
# Choose the appropriate function to read `../data/cdystonia.csv` into a variable called `cdystonia`
# Print the first 15 lines of the `DataFrame` `cdystonia`

cdystonia = pd.read_csv("../data/cdystonia.csv")
cdystonia.head(15)

,patient,obs,week,site,id,treat,age,sex,twstrs
0,1,1,0,1,1,5000U,65,F,32
1,1,2,2,1,1,5000U,65,F,30
2,1,3,4,1,1,5000U,65,F,24
3,1,4,8,1,1,5000U,65,F,37
4,1,5,12,1,1,5000U,65,F,39
5,1,6,16,1,1,5000U,65,F,36
6,2,1,0,1,2,10000U,70,F,60
7,2,2,2,1,2,10000U,70,F,26
8,2,3,4,1,2,10000U,70,F,27
9,2,4,8,1,2,10000U,70,F,41


This dataset is from [Statistical Methods for the Analysis of Repeated Measurements](http://www.amazon.com/Statistical-Methods-Analysis-Repeated-Measurements/dp/0387953701) by Charles S. Davis, pp. 161-163 (Springer, 2002). These data are from a multicenter, randomized controlled trial of botulinum toxin type B (BotB) in patients with cervical dystonia from nine U.S. sites.

Patients were randomized to placebo (N=36), 5000 units of BotB (N=36), or 10,000 units of BotB (N=37). The response variable is the total score on Toronto Western Spasmodic Torticollis Rating Scale (TWSTRS), measuring severity, pain, and disability of cervical dystonia (high scores mean more impairment). TWSTRS was measured at baseline (week 0) and weeks 2, 4, 8, 12, 16 after treatment began, so this is a longitudinal study.

1. Since there are repeated measures of each patient, the `patient` column alone cannot be used as an index, because it is not unique. Use some of the columns in the DataFrame to create an index for the data that is unique.

In [29]:
# Choose a column in addition to "patient" so that the two columns have unique values, and set those two to be the index.
# cdystonia["uniqueID"] = cdystonia["patient"].astype(str) + "-" + cdystonia["obs"].astype(str)
cdystonia = cdystonia.set_index(["patient", "obs"])
cdystonia

week  site  id  treat  age sex  twstrs
patient obs                                        
1       1       0     1   1  5000U   65   F      32
        2       2     1   1  5000U   65   F      30
        3       4     1   1  5000U   65   F      24
        4       8     1   1  5000U   65   F      37
        5      12     1   1  5000U   65   F      39
...           ...   ...  ..    ...  ...  ..     ...
109     1       0     9  11  5000U   57   M      53
        2       2     9  11  5000U   57   M      38
        4       8     9  11  5000U   57   M      33
        5      12     9  11  5000U   57   M      36
        6      16     9  11  5000U   57   M      51

[631 rows x 7 columns]

2. The `cdystonia` dataset is stored in **long** format, meaning that each row contains a single observation. Use pandas functions and methods to change the data to **wide** format, where each row represents the data for a single patient. 

In [30]:
# Convert the data to a wide format, where each row represents the data for a single patient
cdystonia = cdystonia.pivot_table(values='twstrs', index=['patient', 'treat', 'age', 'sex'], columns=['week'])
cdystonia

,,,week,0,2,4,8,12,16
patient,treat,age,sex,,,,,,
1,5000U,65,F,32.0,30.0,24.0,37.0,39.0,36.0
2,10000U,70,F,60.0,26.0,27.0,41.0,65.0,67.0
3,5000U,64,F,44.0,20.0,23.0,26.0,35.0,35.0
4,Placebo,59,F,53.0,61.0,64.0,62.0,NaN,NaN
5,10000U,76,F,53.0,35.0,48.0,49.0,41.0,51.0
...,...,...,...,...,...,...,...,...,...
105,Placebo,79,F,45.0,46.0,33.0,44.0,46.0,48.0
106,5000U,43,M,67.0,63.0,71.0,66.0,68.0,71.0
107,10000U,50,M,57.0,NaN,36.0,23.0,NaN,52.0


3. To get an idea about the efficacy of the treatment, use `pandas` to group the data by treatment group, and calculate the mean and standard deviation of the `twstrs` outcome variable for each group in week 4.

In [31]:
# group the data by treatment group, and calculate the mean and standard deviation of the `twstrs` outcome variable for each group in week 4.
# As a sanity check, this should be the answer here:

#                mean       std
# treat
# 10000U         34.805556  12.188565
# 5000U          37.114286  15.311993
# Placebo        39.342857  11.827045

cdystonia.groupby(["treat"])[4].mean()

treat
10000U     34.805556
5000U      37.114286
Placebo    39.342857
Name: 4, dtype: float64

In [32]:
cdystonia.groupby(["treat"])[4].std()

treat
10000U     12.188565
5000U      15.311993
Placebo    11.827045
Name: 4, dtype: float64

4. Since this is a longitudinal study, graphics are helpful for understanding the dynamics of the experiment. Using the plotting package of your choice (there are many for Python!) create a set of plots showing how the response variable changes over time for each experimental group. 

In [ ]:
# create a set of plots showing how the response variable changes over time for each experimental group


## Linear Algebra


Operations become much faster when we can express them as vectorized linear algebra commands. One example is generating multivariate normal distributions.

Suppose we wish to generate samples with mean $\mu$ and covariance $\Sigma$, where $\mu$ is $n \times 1$ and $\Sigma$ is $n \times n$. One way to do this is to calculate a [*Cholesky decomposition*](https://en.wikipedia.org/wiki/Cholesky_decomposition) of $\Sigma$, so that
$$
\Sigma = LL^T.
$$

Then if $x$ is $n$ independent draws from a standard normal distribution, 
$$
\nu = Lx + \mu \sim \mathcal{N}(\mu, \Sigma)
$$

### Exercises
Let $\mu = (3, 2)$, and
$$
\Sigma = \left(\begin{array}{cc}
1 & 0.9 \\
0.9 & 1
\end{array}\right)
$$

1. Use `np.linalg.cholesky` to compute $L$, the Cholesky decomposition of $\Sigma$
2. In newer Python (3.6+), `@` is matrix multiplication. Confirm that $\Sigma = LL^T$.
3. Draw 2 independent draws from a standard normal using `x = np.random.randn(2)`, and compute $\nu$ using `L @ x + mu`
4. The above exercise generates 1 draw from a multivariate normal. Use `x = np.random.randn(2, 1_000)` and the same formula as above to generate 1,000 draws.

In [ ]:
mu = np.array([[3], [2]])
sigma = np.array([[1, 0.9], [0.9, 1]])

In [ ]:
# Use `np.linalg.cholesky` to compute $L$, the Cholesky decomposition of `sigma`

In [ ]:
# In newer Python (3.7+), `@` is matrix multiplication. Confirm that `sigma = L @ L.T`

In [ ]:
# Draw 2 independent draws from a standard normal using `x = np.random.randn(2)`, and compute `nu` using `L @ x + mu`

In [ ]:
# The above exercise generates 1 draw from a multivariate normal. Use `x = np.random.randn(2, 1_000)` and the same formula as above to generate 1,000 draws

## Probability Distributions and Simulation

Bayesian inference relies on the use of probability distributions for constructing models. Though several statistical and machine learning packages implement their own set of probability distributions, the NumPy and SciPy libraries include general-purpose functions and classes for performing probability operations. 

NumPy has an efficient set of random number generators for different distributions, while SciPy implements a large set of complete probability distributions that allow them to be used in most applications.

In [33]:
from scipy.stats import distributions as dists

In [34]:
dir(dists)[-20:]

['truncnorm_gen',
 'tukeylambda',
 'tukeylambda_gen',
 'uniform',
 'uniform_gen',
 'vonmises',
 'vonmises_gen',
 'vonmises_line',
 'wald',
 'wald_gen',
 'weibull_max',
 'weibull_max_gen',
 'weibull_min',
 'weibull_min_gen',
 'wrapcauchy',
 'wrapcauchy_gen',
 'yulesimon',
 'yulesimon_gen',
 'zipf',
 'zipf_gen']

Let's look at a gamma distribution as an example:

In [35]:
gamma_dist = dists.gamma

Inspecting the attributes of `gamma_dist`, we can see several important methods, including `pdf` (probability distribution function), `cdf` (cumulative distribution function), `rvs` (random number generator), and more.

In [36]:
[f for f in dir(gamma_dist) if not f.startswith('_')]

['a',
 'b',
 'badvalue',
 'cdf',
 'entropy',
 'expect',
 'extradoc',
 'fit',
 'fit_loc_scale',
 'freeze',
 'generic_moment',
 'interval',
 'isf',
 'logcdf',
 'logpdf',
 'logsf',
 'mean',
 'median',
 'moment',
 'moment_type',
 'name',
 'nnlf',
 'numargs',
 'pdf',
 'ppf',
 'random_state',
 'rvs',
 'sf',
 'shapes',
 'stats',
 'std',
 'support',
 'var',
 'vecentropy',
 'xtol']

### Exercise: Simulating linear regression

1. Generate a (1000, 10) `features` array using a uniform distribution.
2. Generate a (10, 1) `weights` array, using a normal distribution with standard deviation 2 and mean 0. Note this is typically unobserved.
3. Generate a (1000, 1) `noise` array, using a normal distribution with standard deviation 1 and mean 0. Note that this is typically unobserved.
4. Compute (1000, 1) `target` array, as `features @ weights + noise`
5. Recover an estimate for the weights using `np.linalg.pinv(features) @ target` (`pinv` is the [Moore-Penrose pseudoinverse](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse))
6. Our statistical model is 
$$
\text{weights} \sim \mathcal{N}(0, 2) \\
\text{target} | \text{features}, \text{weights} \sim \mathcal{N}(\text{features} \cdot \text{weights}, 1)
$$
What is the log probability that all the weights are 0, and all the targets are 1, given your generated `features`?
7. What is the log probability of your generated `weights` and generated `targets`, given your generated `features`?

In [53]:
import numpy as np

In [48]:
# Generate a (1000, 10) `features` array using a uniform distribution.

# asseart features.shape == (1000, 10)

features = dists.uniform()
[f for f in dir(features) if not f.startswith('_')]

<bound method rv_frozen.pdf of <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000264A9F572C8>>

In [59]:
# Generate a (10, 1) `weights` array, using a normal distribution with standard deviation 2 and mean 0
weights = np.random.normal(loc = 0, scale = 2, size = (10, 1))
# assert weights.shape == (10, 1)
print(weights.shape == (10, 1))
weights

True


array([[-2.23365969],
       [ 2.49437507],
       [-3.15705555],
       [ 0.1973058 ],
       [-2.97825567],
       [-0.72710156],
       [ 2.99871089],
       [ 0.60211405],
       [-2.23429698],
       [ 1.8958197 ]])

In [ ]:
# Generate a (1000, 1) `noise` array, using a normal distribution with standard deviation 1 and mean 0

# assert noise.shape == (1000, 1)

In [ ]:
# Compute (1000, 1) `target` array, as `features @ weights + noise`

# assert target.shape == (1000, 1)

In [ ]:
# Recover an estimate for the weights using `np.linalg.pinv(features) @ target`

In [ ]:
# Compute the log probability that all the weights are 0, and all the targets are 1, given your generated `features`

In [ ]:
# Compute the log probability of your generated `weights` and `targets`, given your generated `features`

Your answer to 5 above should match the answer `scikit-learn` provides:

In [37]:
# from sklearn.linear_model import LinearRegression

# LinearRegression(fit_intercept=False).fit(features, target).coef_.T

NameError: name 'features' is not defined

## Optimisation

Another way of running linear regression is via optimization. Suppose we want to minimize the sum of squares using scipy. We can do this using `scipy.optimize`.

In [ ]:
from scipy.optimize import fmin_bfgs

features = np.random.uniform(size=(1000, 10))
weights = np.random.normal(0, 2, size=(10, 1))
noise = np.random.normal(0, 1, size=(1000, 1))
target = (features @ weights + noise).flatten()


### Exercise

Find the weights with the least mean squared error using `fmin_bfgs`. It expects a function to minimize and an initial point.

1. The function to minimize should accept an argument `x`, and return `((features @ x - target) ** 2).mean()`
2. You can initialize with a vector of 10 zeros.

In [ ]:
# Write the function to minimize, and an initial point

def loss_function(x):
    pass

# initial_point = 

# assert initial_point.shape == (10,)

In [ ]:
# min_x = fmin_bfgs(loss_function, initial_point)
# min_x

This answer should match our solution using linear algebra, or using scikit-learn:

In [ ]:
np.linalg.pinv(features) @ target

In [ ]:
from sklearn.linear_model import LinearRegression

LinearRegression(fit_intercept=False).fit(features, target).coef_.T

## Theano

Our primary tool for building Bayesian models in this course is PyMC3, which relies on a third-party computational backend called Theano for much of the heavy-lifting. Theano was created to perform deep learning, but PyMC3 uses some of its key features to implement modern sampling-based fitting methods. This section is a short introduction to Theano to give you a little practice with some of its data structures and functions before we jump in and use PyMC3.

### Performing operations on data structures

To get us started with Theano and get a feel of what we're working with, 
let's make a simple function: multiply a scalar with a matrix, and add that to another matrix. Here is how you do
it:

#### Step 1 - Declaring Variables

In [60]:
import numpy as np

from theano import function, shared
import theano.tensor as tt
import theano

c = tt.dscalar('c')
x = tt.dmatrix('x')
y = tt.dmatrix('y')

In Theano, all symbols must be typed. In particular, `tt.dscalar`
is the type we assign to "0-dimensional arrays (`scalar`) of doubles
(`d`). It is a Theano `type`.

In [61]:
type(c)

theano.tensor.var.TensorVariable

``dmatrix`` is the Type for matrices of doubles. Then we can use
our new function on 2D arrays.


In [ ]:
x.type

In [ ]:
tt.dscalar

Notice that we have created objects of the type `TensorVariable`. A **tensor** is a generalization of an array to (potentially) multiple dimensions. Thus, everything from a scalar to a 5-dimensional hyper-matrix can be accomodated with the same abstraction. All expressions defined in Theano are performed by associating tensors with operations and with one another.

### Step 2 - Symbolic Expressions

The second step is to multiply `c` with `x`, then add `y`, yielding another variable `z`:

In [ ]:
z = c*x + y

You can use the `pp` function to *pretty-print* out the computation associated to *z*.


In [ ]:
from theano.printing import pp
print(pp(z))

### Step 3 - Compiling a Function

The last step is to create a function taking `c`, `x` and `y` as inputs
and giving `z` as output:

In [ ]:
f = function([c, x, y], z)

The first argument to `function()` is a list of Variables
that will be provided as inputs to the function. The second argument
is a single Variable *or* a list of Variables. For either case, the second
argument is what we want to see as output when we apply the function. *f* may
then be used like a normal Python function.


Now we can call the function:

In [ ]:
C = 2
X = [[1, 2], [3, 4]]
Y = [[10, 20], [30, 40]]

f(C, X, Y)

If you are following along and typing into an interpreter, you may have
noticed that there was a slight delay in executing the ``function``
instruction. Behind the scenes, *f* was being **compiled into C code**.

Internally, Theano builds a graph structure composed of interconnected `Variable` nodes, `op` nodes and `apply` nodes. 

An `op` node encapsulates a particular mathematical operation, such as an arithmetic operation or a transformation.

An `apply` node represents the application of an `op` to some variables. It is important to draw the difference between the definition of a computation represented by an `op` and its application to some actual data which is represented by the apply node. 

In [ ]:
from IPython.display import Image

Image(theano.printing.pydotprint(f, return_image=True))

A `Variable` is the main data structure you work with when
using Theano. By calling `tt.dscalar` or `tt.dmatrix` with a string argument, you create a
`Variable` representing a floating-point scalar or matrix quantity with the
given name. If you provide no argument, the symbol will be unnamed. Names
are not required, but they can help debugging.

### Exercise

Let's create a (slightly) more interesting function: a logistic curve. The logistic transformation is:

$$\log\left[\frac{p}{1-p}\right] = x$$

Which transforms values on the $[0, 1]$i interval to the real line. Most often, we are doing the inverse of this, for example, when we build a logistic regression. The inverse of this function is:

$$p = \frac{1}{1 + \exp(-x)}$$

Implement this function in Theano, compile it, and run it on one of the matrices created above.

In [ ]:
# Hint: most of Theano's functions are in the theano.tensor submodule.